# Введение в MapReduce модель на Python


In [8]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [9]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [10]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [11]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [12]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [13]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [14]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [15]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [16]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [17]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [18]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [19]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [20]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [21]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [22]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.232319248988338),
 (1, 1.232319248988338),
 (2, 1.232319248988338),
 (3, 1.232319248988338),
 (4, 1.232319248988338)]

## Inverted index 

In [23]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [24]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [25]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [26]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('what', 10)]),
 (1, [('a', 2), ('banana', 2), ('is', 18), ('it', 18)])]

## TeraSort

In [27]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.06541533234668384),
   (None, 0.10198879314843423),
   (None, 0.11583145896736213),
   (None, 0.22023053253175662),
   (None, 0.2630642142888373),
   (None, 0.2693043938078179),
   (None, 0.3354924177543577),
   (None, 0.4070553080347965),
   (None, 0.41963720037608665),
   (None, 0.4231451675736174),
   (None, 0.43164947830421796),
   (None, 0.4357987993340934),
   (None, 0.47098715819013215),
   (None, 0.4838982126729885)]),
 (1,
  [(None, 0.5066609854666776),
   (None, 0.5411986256138147),
   (None, 0.5798491803152442),
   (None, 0.5829963061123334),
   (None, 0.5960590031032266),
   (None, 0.6178199295659546),
   (None, 0.6613010193137099),
   (None, 0.7724544564699329),
   (None, 0.7898604580456244),
   (None, 0.7975191958004487),
   (None, 0.8004200688423206),
   (None, 0.8111704844069557),
   (None, 0.8310298391825287),
   (None, 0.8892086575891816),
   (None, 0.9158778892345327),
   (None, 0.9245558147612012)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [28]:
import random
from collections import defaultdict

def mapper(numbers):
    for number in numbers:
        yield (None, number)
        
def reducer(key, values):
    return max(values)

def map_reduce(numbers):
    mapped = []
    for number in numbers:
        mapped.append((None, number))

    
    grouped = defaultdict(list)
    for key, value in mapped:
        grouped[key].append(value)
    
    results = {}
    for key, values in grouped.items():
        results[key] = reducer(key, values)
    
    return results

numbers = [random.randint(0, 1000000000) for i in range(1000000)]
# Пример использования
result = map_reduce(numbers)
max_value = result[None]
print("Максимальное число:", max_value)


Максимальное число: 999996753


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [29]:
def mapper(numbers):
    for number in numbers:
        yield (None, (number, 1))  

def reducer(key, values):
    total_sum = 0
    count = 0
    for value in values:
        total_sum += value[0]  # Суммируем числа
        count += value[1]      # Считаем количество
    return total_sum, count

def map_reduce(numbers):
    mapped = []
    for number in numbers:
        mapped.append((None, (number, 1))) 
    
    grouped = defaultdict(list)
    for key, value in mapped:
        grouped[key].append(value)  
    
    results = {}
    for key, values in grouped.items():
        results[key] = reducer(key, values)  
    
    return results

numbers = [random.randint(0, 1000000000) for k in range(1000000)]

# Пример использования
result = map_reduce(numbers)
total_sum, count = result[None]

if count > 0:
    average = total_sum / count  # Вычисляем среднее
else:
    average = 0

print("Арифметическое среднее:", average)


Арифметическое среднее: 500222407.350149


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [30]:
def groupbykey(iterable):
    # Сортируем входные данные по ключу
    sorted_iterable = sorted(iterable, key=lambda x: x[0])
    
    grouped = {}
    
    for (k2, v2) in sorted_iterable:
        # Добавляем значение к соответствующему ключу
        if k2 not in grouped:
            grouped[k2] = []
        grouped[k2].append(v2)
    
    return grouped.items()

# Примеры использования
data1 = [
    (1, 'a'),
    (2, 'b'),
    (1, 'c'),
    (3, 'd'),
    (2, 'e'),
]

data2 = [
    (4, 'x'),
    (2, 'y'),
    (4, 'z'),
    (1, 'w'),
    (2, 'v'),
]

print("Результат для data1:")
print(dict(groupbykey(data1)))  

print("\nРезультат для data2:")
print(dict(groupbykey(data2)))

Результат для data1:
{1: ['a', 'c'], 2: ['b', 'e'], 3: ['d']}

Результат для data2:
{1: ['w'], 2: ['y', 'v'], 4: ['x', 'z']}


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [31]:
from typing import NamedTuple, Iterator, List, Set, Dict
# В данной задаче исключаем дубликаты для полей - год, социальные контакты и пол.
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

def MAP(_, row: User) -> Iterator:
    # Возвращаем все пользователи без фильтрации
    yield (row.id, row)

def RECORDREADER() -> List[User]:
    return [
        User(id=0, age='55', gender='male', social_contacts=20),
        User(id=1, age='25', gender='female', social_contacts=240),
        User(id=2, age='25', gender='female', social_contacts=500),
        User(id=3, age='33', gender='female', social_contacts=800),
        User(id=4, age='25', gender='female', social_contacts=240)  # Дубликат по всем полям, кроме id
    ]

def drop_duplicates(users: List[User]) -> Set[User]:
    # Используем словарь для исключения дубликатов по всем полям, кроме id
    unique_users = {}
    for user in users:
        # Создаем ключ на основе всех полей, кроме id
        key = (user.age, user.social_contacts, user.gender)
        unique_users[key] = user  # Сохраняем пользователя по этому ключу

    return set(unique_users.values()), unique_users

def find_non_unique(users: List[User]) -> List[User]:
    # Используем словарь для подсчета количества вхождений пользователей
    user_count = {}
    for user in users:
        key = (user.age, user.social_contacts, user.gender)
        if key in user_count:
            user_count[key].append(user)
        else:
            user_count[key] = [user]

    # Извлекаем только тех пользователей, которые встречаются более одного раза
    non_unique_users = [user for user_list in user_count.values() if len(user_list) > 1 for user in user_list]

    return non_unique_users

def distributed_drop_duplicates(input_data: List[User]) -> Set[User]:
    # Шаг 1: Применяем MAP к каждому элементу входной коллекции
    mapped_results = [list(MAP(None, user)) for user in input_data]
    
    # Извлекаем только значения (вторые элементы кортежей)
    filtered_users = [user for sublist in mapped_results for _, user in sublist]

    # Шаг 2: Удаляем дубликаты
    unique_users, _ = drop_duplicates(filtered_users)

    return unique_users

# Подготовка данных
input_collection = RECORDREADER()

# Выполнение распределенной операции исключения дубликатов
result = distributed_drop_duplicates(input_collection)

# Находим не уникальные элементы
non_unique_result = find_non_unique(input_collection)

# Проверка работы функции
print("Отсортированные данные:")
for user in result:
    print(user)

print("\nЭлементы с совпадениями:")
for user in non_unique_result:
    print(user)


Отсортированные данные:
User(id=4, age='25', social_contacts=240, gender='female')
User(id=2, age='25', social_contacts=500, gender='female')
User(id=3, age='33', social_contacts=800, gender='female')
User(id=0, age='55', social_contacts=20, gender='male')

Элементы с совпадениями:
User(id=1, age='25', social_contacts=240, gender='female')
User(id=4, age='25', social_contacts=240, gender='female')


### Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [32]:
from collections import defaultdict
import random

def generate_records(count):
    """Создает список записей с случайными значениями."""
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def filter_even(t):
    """Проверяет, является ли первый элемент кортежа четным."""
    return t[0] % 2 == 0

def map_records(el_list):
    """Отображает записи, фильтруя их по условию."""
    mapped_result = defaultdict(list)
    for t in el_list:
        if filter_even(t):
            mapped_result[t].append(t)
    return mapped_result.items()

def reduce_mapped_items(mapped_items):
    """Сводит отображенные элементы в один список."""
    reduced_result = []
    for values in mapped_items:
        for value in values:
            reduced_result.append(value)
    return reduced_result

def partition_records(records, part_size):
    """Разделяет записи на части заданного размера."""
    return [records[i:i + part_size] for i in range(0, len(records), part_size)]

# Основной код
if __name__ == "__main__":
    record_count = 5
    records = generate_records(record_count)
    
    print("Сгенерированные записи:")
    print(records)

    partition_size = 5
    record_partitioned = partition_records(records, partition_size)
    
    print("Записи, разделенные на части:")
    print(record_partitioned)

    # Применяем MAP и REDUCE
    mapped_results = list(map(map_records, record_partitioned))
    
    print("Результаты отображения:")
    print(mapped_results)

    reduced_results = reduce_mapped_items(mapped_results)
    
    print("Сведенные результаты:")
    print(reduced_results)


Сгенерированные записи:
[(2, 60, 100), (62, 11, 64), (46, 76, 13), (14, 8, 15), (5, 62, 44)]
Записи, разделенные на части:
[[(2, 60, 100), (62, 11, 64), (46, 76, 13), (14, 8, 15), (5, 62, 44)]]
Результаты отображения:
[dict_items([((2, 60, 100), [(2, 60, 100)]), ((62, 11, 64), [(62, 11, 64)]), ((46, 76, 13), [(46, 76, 13)]), ((14, 8, 15), [(14, 8, 15)])])]
Сведенные результаты:
[((2, 60, 100), [(2, 60, 100)]), ((62, 11, 64), [(62, 11, 64)]), ((46, 76, 13), [(46, 76, 13)]), ((14, 8, 15), [(14, 8, 15)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [33]:
import random
from typing import Iterator, NamedTuple, List, Tuple

# Константа для хранения значений
S = {148, 66, 63}

def MAP(t: Tuple[int, int, int]) -> Tuple[Tuple[int, int, int], Tuple[int, int, int]]:
    """Функция MAP фильтрует элементы кортежа t по наличию в множестве S."""
    res_list = [el for el in t if el in S]
    return (tuple(res_list), tuple(res_list))

def REDUCE(key: int, values: Iterator[NamedTuple]) -> Tuple[int, int]:
    """Функция REDUCE возвращает ключ и его значение."""
    return (key, key)

def RECORDREADER(count: int) -> List[Tuple[int, int, int]]:
    """Создает список записей с случайными значениями."""
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable: List[Tuple[int, Tuple[int, int]]]) -> List[Tuple[int, List[Tuple[int, int]]]]:
    """Группирует элементы по ключу."""
    grouped = {}
    for (k2, v2) in iterable:
        grouped[k2] = grouped.get(k2, []) + [v2]
    return list(grouped.items())

def main():
    # Генерация записей
    record_count = 5
    records = RECORDREADER(record_count)
    print("Сгенерированные записи:", records)

    # Применение MAP
    map_output = list(map(MAP, RECORDREADER(100)))
    print("MAP:", map_output)

    # Shuffle (группировка по ключу)
    shuffle_output = group_by_key(map_output)
    print("Shuffle:", shuffle_output)

    # Применение REDUCE
    reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
    print("Reduce:", reduce_output)

if __name__ == "__main__":
    main()


Сгенерированные записи: [(50, 19, 7), (53, 55, 83), (34, 22, 71), (47, 84, 43), (57, 65, 59)]
MAP: [((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((66,), (66,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((63,), (63,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ())

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [34]:
import random
from typing import Iterator, NamedTuple, List, Tuple

def MAP(t: Tuple[int, int, int]) -> Tuple[Tuple[int, int, int], Tuple[int, int, int]]:
    """Функция MAP возвращает входной кортеж дважды."""
    return (t, t)

def REDUCE(key: int, values: Iterator[NamedTuple]) -> Tuple[int, int]:
    """Функция REDUCE возвращает ключ и его значение."""
    return (key, key)

def RECORDREADER(count: int) -> List[Tuple[int, int, int]]:
    """Создает список записей с случайными значениями."""
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable: List[Tuple[int, Tuple[int, int]]]) -> List[Tuple[int, List[Tuple[int, int]]]]:
    """Группирует элементы по ключу."""
    grouped = {}
    for (k2, v2) in iterable:
        grouped[k2] = grouped.get(k2, []) + [v2]
    return list(grouped.items())

def main():
    # Генерация записей
    record_count = 5
    records = RECORDREADER(record_count)
    print("Сгенерированные записи:", records)

    # Применение MAP
    map_output = list(map(MAP, RECORDREADER(100)))
    print("MAP:", map_output)

    # Shuffle (группировка по ключу)
    shuffle_output = group_by_key(map_output)
    print("Shuffle:", shuffle_output)

    # Применение REDUCE
    reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
    print("Reduce:", reduce_output)

if __name__ == "__main__":
    main()


Сгенерированные записи: [(64, 52, 97), (45, 6, 81), (98, 3, 51), (39, 32, 90), (14, 44, 39)]
MAP: [((95, 10, 41), (95, 10, 41)), ((4, 24, 64), (4, 24, 64)), ((46, 92, 96), (46, 92, 96)), ((59, 20, 21), (59, 20, 21)), ((86, 98, 69), (86, 98, 69)), ((83, 22, 96), (83, 22, 96)), ((82, 11, 82), (82, 11, 82)), ((100, 44, 2), (100, 44, 2)), ((87, 18, 39), (87, 18, 39)), ((19, 9, 94), (19, 9, 94)), ((50, 5, 30), (50, 5, 30)), ((77, 33, 85), (77, 33, 85)), ((86, 46, 33), (86, 46, 33)), ((45, 19, 22), (45, 19, 22)), ((84, 32, 61), (84, 32, 61)), ((2, 58, 25), (2, 58, 25)), ((99, 65, 75), (99, 65, 75)), ((18, 44, 41), (18, 44, 41)), ((57, 63, 60), (57, 63, 60)), ((69, 79, 9), (69, 79, 9)), ((2, 62, 68), (2, 62, 68)), ((3, 32, 59), (3, 32, 59)), ((10, 2, 32), (10, 2, 32)), ((67, 100, 65), (67, 100, 65)), ((15, 1, 47), (15, 1, 47)), ((3, 88, 41), (3, 88, 41)), ((19, 73, 0), (19, 73, 0)), ((63, 73, 85), (63, 73, 85)), ((58, 30, 29), (58, 30, 29)), ((86, 63, 19), (86, 63, 19)), ((64, 83, 34), (64, 8

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [35]:
import random
from typing import Iterator, NamedTuple, List, Tuple

def MAP(t: Tuple[int, int]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """Функция MAP возвращает входной кортеж дважды."""
    return (t, t)

def REDUCE(key: int, values: List[int]) -> Tuple[int, int]:
    """Функция REDUCE возвращает ключ и его значение, если длина значений равна 2."""
    if len(values) == 2:
        return (key, key)
    return None

def RECORDREADER(count: int) -> List[Tuple[int, int]]:
    """Создает список записей с случайными значениями."""
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def group_by_key(iterable: List[Tuple[int, int]]) -> List[Tuple[int, List[int]]]:
    """Группирует элементы по ключу."""
    grouped = {}
    for (k2, v2) in iterable:
        grouped[k2] = grouped.get(k2, []) + [v2]
    return list(grouped.items())

def main():
    # Генерация записей
    record_count = 100
    records = RECORDREADER(record_count)
    print("Сгенерированные записи:", records)

    # Применение MAP
    map_output = list(map(MAP, records))
    print("MAP:", map_output)

    # Shuffle (группировка по ключу)
    shuffle_output = group_by_key(map_output)
    print("Shuffle:", shuffle_output)

    # Применение REDUCE
    reduce_output = [el[0] for el in map(lambda x: REDUCE(*x), shuffle_output) if el is not None]
    print("Reduce:", reduce_output)

if __name__ == "__main__":
    main()


Сгенерированные записи: [(0, 0), (0, 1), (1, 3), (0, 3), (3, 1), (2, 2), (1, 1), (0, 2), (1, 0), (2, 1), (3, 1), (3, 1), (1, 1), (2, 3), (0, 3), (1, 3), (3, 0), (0, 2), (3, 1), (2, 0), (0, 2), (1, 2), (0, 3), (3, 1), (2, 3), (2, 2), (1, 3), (0, 0), (1, 2), (2, 3), (0, 0), (2, 2), (3, 2), (1, 0), (1, 3), (0, 1), (2, 1), (3, 1), (3, 3), (0, 2), (1, 1), (2, 3), (1, 1), (0, 1), (0, 1), (1, 1), (0, 2), (1, 0), (1, 1), (3, 3), (2, 0), (0, 2), (3, 1), (0, 1), (3, 3), (2, 3), (2, 2), (1, 1), (2, 2), (3, 1), (2, 0), (2, 1), (1, 1), (0, 1), (2, 3), (1, 3), (3, 3), (0, 0), (0, 3), (3, 3), (3, 3), (0, 2), (3, 1), (1, 1), (0, 2), (3, 1), (1, 1), (3, 2), (2, 3), (0, 1), (2, 1), (0, 2), (0, 2), (0, 3), (3, 2), (3, 0), (1, 2), (2, 3), (3, 3), (0, 3), (1, 2), (3, 1), (3, 0), (2, 1), (1, 1), (0, 3), (0, 3), (3, 1), (1, 2), (3, 1)]
MAP: [((0, 0), (0, 0)), ((0, 1), (0, 1)), ((1, 3), (1, 3)), ((0, 3), (0, 3)), ((3, 1), (3, 1)), ((2, 2), (2, 2)), ((1, 1), (1, 1)), ((0, 2), (0, 2)), ((1, 0), (1, 0)), ((2, 1)

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [36]:
import random
from typing import List, Tuple as PyTuple, Iterator, NamedTuple

# Определяем классы и функции
class Tuple:
    def __init__(self, data: PyTuple[int, int], rel_id: int):
        self.data = data
        self.rel_id = rel_id

# Генерация случайного кортежа
def get_random_tuple(count: int) -> Tuple:
    data = tuple((random.randint(0, 3), random.randint(0, 3)) for _ in range(count))
    rel_id = rels[random.randint(0, len(rels) - 1)]
    return Tuple(data, rel_id)

# Чтение записей
def RECORDREADER(count: int) -> List[Tuple]:
    return [get_random_tuple(3) for _ in range(count)]

# Функция MAP
def MAP(t: Tuple) -> PyTuple[PyTuple[int, int], int]:
    return (t.data, t.rel_id)

# Функция REDUCE
def REDUCE(key: int, values: List[int]) -> PyTuple[int, int]:
    if values == [rels[0]]:
        return (key, key)
    return None

# Группировка по ключу
def group_by_key(iterable: List[PyTuple[PyTuple[int, int], int]]) -> dict:
    grouped = {}
    for (k2, v2) in iterable:
        grouped[k2] = grouped.get(k2, []) + [v2]
    return grouped

def main():
    # Инициализация переменных
    global rels
    rels = [1, 2]

    # Генерация записей
    record_count = 100
    records = RECORDREADER(record_count)

    # Применение MAP
    map_out = list(map(MAP, records))
    print("MAP:", map_out)

    # Shuffle (группировка по ключу)
    shuffle_out = group_by_key(map_out)
    print("Shuffle:", list(shuffle_out.items()))

    # Применение REDUCE
    reduce_out = [el[0] for el in (map(lambda x: REDUCE(*x), shuffle_out.items())) if el is not None]
    print("Reduce:", reduce_out)

if __name__ == "__main__":
    main()


MAP: [(((2, 3), (0, 3), (0, 0)), 2), (((0, 0), (1, 2), (2, 0)), 1), (((1, 2), (1, 0), (3, 3)), 1), (((2, 3), (0, 2), (0, 3)), 1), (((2, 1), (3, 2), (3, 3)), 1), (((1, 1), (3, 0), (3, 1)), 2), (((0, 1), (1, 1), (3, 0)), 2), (((3, 1), (0, 1), (1, 2)), 1), (((3, 3), (1, 0), (3, 3)), 1), (((3, 0), (0, 2), (2, 3)), 2), (((3, 1), (1, 0), (2, 2)), 1), (((0, 2), (3, 3), (2, 0)), 1), (((2, 3), (0, 3), (2, 2)), 2), (((1, 1), (0, 1), (1, 1)), 2), (((2, 0), (0, 3), (3, 2)), 1), (((3, 0), (0, 1), (3, 2)), 2), (((3, 0), (0, 2), (2, 1)), 2), (((3, 3), (3, 1), (1, 1)), 1), (((1, 0), (2, 0), (3, 0)), 2), (((0, 1), (0, 1), (1, 1)), 1), (((0, 2), (3, 0), (1, 3)), 1), (((3, 0), (3, 1), (2, 2)), 2), (((0, 1), (1, 1), (1, 2)), 2), (((0, 0), (3, 3), (1, 1)), 1), (((2, 1), (1, 3), (3, 3)), 1), (((2, 2), (2, 0), (1, 1)), 1), (((3, 1), (1, 2), (3, 3)), 1), (((3, 3), (3, 3), (0, 0)), 1), (((2, 0), (3, 0), (1, 1)), 1), (((0, 3), (1, 0), (1, 2)), 2), (((0, 0), (2, 3), (2, 3)), 1), (((2, 3), (0, 3), (2, 2)), 1), ((

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [37]:
import random
from typing import List, Tuple as PyTuple, Iterator, NamedTuple

# Глобальная переменная для отношений
rels = [1, 2]

# Определение класса Tuple
class CustomTuple:
    def __init__(self, data: PyTuple[int, int], rel_id: int):
        self.data = data
        self.rel_id = rel_id

# Генерация случайного кортежа
def get_random_tuple() -> CustomTuple:
    data = (random.randint(0, 3), random.randint(0, 3))
    rel_id = rels[random.randint(0, len(rels) - 1)]
    return CustomTuple(data, rel_id)

# Чтение записей
def RECORDREADER(count: int) -> List[CustomTuple]:
    return [get_random_tuple() for _ in range(count)]

# Функция MAP
def MAP(t: CustomTuple) -> PyTuple[int, PyTuple[int, int]]:
    if t.rel_id == rels[0]:
        return (t.data[1], (t.rel_id, t.data[0]))
    else:
        return (t.data[0], (t.rel_id, t.data[1]))

# Функция REDUCE
def REDUCE(key: int, values: Iterator[PyTuple[int, int]]) -> List[PyTuple[int, int, int]]:
    res = []
    for v in values:
        res.append((v[0], key, v[1]))
    return res

# Группировка по ключу
def group_by_key(iterable: List[PyTuple[int, PyTuple[int, int]]]) -> dict:
    grouped = {}
    for (k2, v2) in iterable:
        grouped[k2] = grouped.get(k2, []) + [v2]
    return grouped

def main():
    # Генерация записей
    record_count = 100
    records = RECORDREADER(record_count)

    # Применение MAP
    map_out = list(map(MAP, records))
    print("MAP:", map_out)

    # Shuffle (группировка по ключу)
    shuffle_out = group_by_key(map_out)
    print("Shuffle:", list(shuffle_out.items()))

    # Применение REDUCE
    reduce_out = [REDUCE(k, v) for k, v in shuffle_out.items()]
    print("Reduce:", reduce_out)

if __name__ == "__main__":
    main()


MAP: [(0, (2, 1)), (2, (1, 0)), (0, (1, 3)), (1, (1, 3)), (2, (2, 1)), (0, (2, 0)), (1, (1, 1)), (0, (1, 1)), (2, (2, 2)), (0, (2, 2)), (2, (2, 3)), (1, (1, 2)), (0, (2, 2)), (0, (2, 3)), (0, (2, 2)), (3, (1, 3)), (1, (1, 1)), (1, (1, 1)), (0, (2, 2)), (1, (2, 2)), (1, (2, 2)), (1, (1, 3)), (2, (2, 3)), (2, (2, 0)), (0, (2, 2)), (3, (1, 3)), (3, (2, 1)), (3, (1, 2)), (1, (1, 2)), (0, (1, 0)), (3, (2, 0)), (3, (2, 3)), (3, (2, 1)), (0, (2, 2)), (3, (1, 0)), (3, (1, 1)), (2, (2, 3)), (1, (1, 3)), (3, (2, 2)), (2, (1, 3)), (1, (2, 3)), (0, (1, 0)), (0, (1, 0)), (1, (1, 0)), (1, (1, 3)), (1, (1, 2)), (0, (2, 1)), (0, (2, 1)), (1, (1, 0)), (3, (2, 2)), (2, (1, 3)), (0, (1, 3)), (1, (2, 3)), (2, (2, 2)), (3, (2, 3)), (1, (1, 1)), (0, (2, 1)), (0, (2, 1)), (0, (2, 1)), (2, (2, 1)), (2, (2, 0)), (1, (2, 2)), (0, (2, 0)), (0, (1, 3)), (0, (1, 0)), (0, (1, 2)), (2, (2, 3)), (2, (2, 0)), (3, (1, 1)), (2, (2, 0)), (3, (2, 2)), (0, (2, 2)), (0, (1, 2)), (2, (1, 3)), (2, (1, 3)), (1, (2, 0)), (2, (2

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [38]:
import random
from typing import List, Tuple, Iterator, Dict, NamedTuple

# Генерация случайного кортежа
def get_random_tup() -> Tuple[int, int, int]:
    return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

# Чтение записей
def RECORDREADER(count: int) -> List[Tuple[int, int, int]]:
    return [get_random_tup() for _ in range(count)]

# Функция MAP
def MAP(t: Tuple[int, int, int]) -> Tuple[int, int]:
    return (t[0], t[1])

# Суммирование значений
def tetta(values: List[int]) -> int:
    return sum(values)

# Функция REDUCE
def REDUCE(key: int, values: Iterator[int]) -> Tuple[int, int]:
    x = tetta(list(values))  # Преобразуем в список для суммирования
    return (key, x)

# Группировка по ключу
def group_by_key(iterable: List[Tuple[int, int]]) -> Dict[int, List[int]]:
    grouped = {}
    for (k2, v2) in iterable:
        grouped[k2] = grouped.get(k2, []) + [v2]
    return grouped

def main():
    # Генерация записей
    record_count = 100
    records = RECORDREADER(record_count)

    # Применение MAP
    map_out = list(map(MAP, records))
    print("MAP:", map_out)

    # Shuffle (группировка по ключу)
    shuffle_out = group_by_key(map_out)
    print("Shuffle:", list(shuffle_out.items()))

    # Применение REDUCE
    reduce_out = [REDUCE(k, v) for k, v in shuffle_out.items()]
    print("Reduce:", reduce_out)

if __name__ == "__main__":
    main()


MAP: [(2, 0), (3, 0), (3, 3), (0, 3), (1, 2), (1, 3), (3, 3), (0, 3), (2, 3), (0, 3), (3, 2), (1, 1), (2, 1), (3, 3), (2, 1), (1, 3), (1, 2), (2, 1), (1, 3), (3, 3), (3, 1), (3, 0), (1, 0), (0, 1), (1, 1), (1, 0), (3, 2), (2, 2), (2, 1), (2, 0), (0, 0), (2, 3), (1, 1), (3, 2), (0, 0), (2, 3), (0, 1), (0, 0), (1, 0), (1, 2), (3, 0), (0, 2), (0, 0), (3, 3), (1, 2), (1, 2), (2, 3), (2, 1), (2, 3), (3, 3), (0, 0), (0, 2), (2, 2), (1, 0), (2, 2), (2, 2), (1, 3), (1, 3), (2, 0), (2, 1), (2, 2), (3, 3), (3, 2), (3, 2), (1, 1), (3, 1), (0, 1), (2, 3), (1, 0), (3, 0), (0, 1), (2, 0), (2, 3), (3, 1), (0, 1), (3, 0), (2, 0), (3, 0), (0, 0), (1, 2), (0, 1), (2, 1), (2, 0), (1, 1), (0, 0), (3, 3), (2, 0), (2, 1), (3, 0), (1, 0), (1, 3), (3, 1), (1, 3), (0, 3), (3, 0), (3, 3), (3, 3), (2, 1), (0, 2), (2, 1)]
Shuffle: [(2, [0, 3, 1, 1, 1, 2, 1, 0, 3, 3, 3, 1, 3, 2, 2, 2, 0, 1, 2, 3, 0, 3, 0, 1, 0, 0, 1, 1, 1]), (3, [0, 3, 3, 2, 3, 3, 1, 0, 2, 2, 0, 3, 3, 3, 2, 2, 1, 0, 1, 0, 0, 3, 0, 1, 0, 3, 3]), (0

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [39]:
import numpy as np
# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

# Dimensions
I = 2  # Number of rows in small_mat
J = 3  # Number of columns in small_mat / rows in big_mat
K = 4 * 10  # Number of columns in big_mat

# Generate random matrices
small_mat = np.random.rand(I, J)  # Small matrix (I x J)
big_mat = np.random.rand(J, K)     # Big matrix (J x K)

def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])

def MAP(k1, v1):
    (j, k) = k1
    w = v1
    # Yielding the contributions to the result from the small matrix
    for i in range(I):
        yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
    (i, k) = key
    # Summing up the contributions for each (i, k)
    yield (key, sum(values))

Проверьте своё решение

In [40]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw
    return mat

# Verify if the solution matches the reference
print(np.allclose(reference_solution, asmatrix(solution)))  # should return true


True


In [41]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [42]:
import numpy as np

# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

# Dimensions
I = 2  # Number of rows in small_mat
J = 3  # Number of columns in small_mat / rows in big_mat
K = 4 * 10  # Number of columns in big_mat

# Generate random matrices
small_mat = np.random.rand(I, J)  # Small matrix (I x J)
big_mat = np.random.rand(J, K)     # Big matrix (J x K)

def RECORDREADER():
    # First yield the small matrix
    for j in range(small_mat.shape[0]):
        for i in range(small_mat.shape[1]):
            yield (('small', j, i), small_mat[j, i])
    
    # Then yield the big matrix
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield (('big', j, k), big_mat[j, k])

def MAP(k1, v1):
    if k1[0] == 'small':
        j, i = k1[1], k1[2]
        # For each element in the small matrix, emit contributions for each column of the big matrix
        for k in range(big_mat.shape[1]):
            yield ((j, k), v1 * big_mat[i, k])  # Emit key as (row_index_of_small_matrix, column_index_of_big_matrix)
    elif k1[0] == 'big':
        j, k = k1[1], k1[2]
        # For big matrix, we do not need to emit anything here
        pass

def REDUCE(key, values):
    (i, k) = key
    # Summing up the contributions for each (i, k)
    yield (key, sum(values))

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


In [43]:
def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.zeros((I, K))  # Initialize with zeros
    for ((i, k), vw) in reduce_output:
        mat[i, k] += vw  # Use += to accumulate values
    return mat


In [44]:
# Verify if the solution matches the reference
print(np.allclose(reference_solution, asmatrix(solution)))  # should return true
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

True


1

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [45]:
import numpy as np

# Matrix
rows_A = 2
cols_A = 3
cols_B = 4 * 10

# Generation
matrix_A = np.random.rand(rows_A, cols_A)
matrix_B = np.random.rand(cols_A, cols_B)


reference_solution = np.matmul(matrix_A, matrix_B)

def flatten(nested_iterable):
    """A function for aligning nested iterable objects."""
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def group_by_key(iterable):
    grouped = {}
    for (key, value) in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

def group_by_key_distributed(map_partitions, partitioner):
    """Distributed grouping by key."""
    global num_reducers
    partitions = [dict() for _ in range(num_reducers)]
    for map_partition in map_partitions:
        for (key, value) in map_partition:
            partition_id = partitions[partitioner(key)]
            partition_id[key] = partition_id.get(key, []) + [value]
    return [(i, sorted(partition.items(), key=lambda x: x[0])) for i, partition in enumerate(partitions)]

def partitioner(obj):
    """Function for determining the batch number."""
    global num_reducers
    return hash(obj) % num_reducers

def MapReduceDistributed(input_format, map_func, reduce_func, partitioner=partitioner, combiner=None):

    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: map_func(*k1v1), record_reader)), input_format())
    if combiner is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: combiner(*k2v2), group_by_key(map_partition))), map_partitions)
    reduce_partitions = group_by_key_distributed(map_partitions, partitioner)
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: reduce_func(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(sum(len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions]))))
    return reduce_outputs

def as_matrix(reduce_output):
    """Converting the reduction output to a matrix."""
    reduce_output = list(reduce_output)
    rows = max(i for ((i, k), vw) in reduce_output) + 1
    cols = max(k for ((i, k), vw) in reduce_output) + 1
    result_matrix = np.empty(shape=(rows, cols))
    
    for ((i, k), vw) in reduce_output:
        result_matrix[i, k] = vw
        
    return result_matrix

def input_format():

# Generating data for the first matrix
    first_matrix_data = []
    for i in range(matrix_A.shape[0]):
        for j in range(matrix_A.shape[1]):
            first_matrix_data.append(((0, i, j), matrix_A[i, j]))
    yield first_matrix_data

# Generating data for the second matrix
    second_matrix_data = []
    for j in range(matrix_B.shape[0]):
        for k in range(matrix_B.shape[1]):
            second_matrix_data.append(((1, j, k), matrix_B[j, k]))
    yield second_matrix_data

def map_join(k1, v1):

    (matrix_id, i, j) = k1
    value = v1
    if matrix_id == 0:
        yield (j, (matrix_id, i, value))
    else:
        yield (i, (matrix_id, j, value))

def reduce_join(key, values):

    from_first_matrix = [v for v in values if v[0] == 0]
    from_second_matrix = [v for v in values if v[0] == 1]
    
    for f in from_first_matrix:
        for s in from_second_matrix:
            yield ((f[1], s[1]), f[2] * s[2])

def get_joined():
    for j in joined_data:
        print("Join Part:", j)
        yield j[1]

def map_multiply(k1, v1):
    yield (k1, v1)

def reduce_multiply(key, values):
    result_value = sum(values)
    yield (key, result_value)

num_maps = 4
num_reducers = 2
partitioned_output = MapReduceDistributed(input_format, map_join, reduce_join)
joined_data = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

print(joined_data)


mul_output = MapReduceDistributed(get_joined, map_multiply, reduce_multiply)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)
solution_values = []
for p in pre_result:
    solution_values.extend(p[1])


print(np.allclose(reference_solution, as_matrix(solution_values)))


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.015389429445823034), ((0, 1), 0.036898521172696805), ((0, 2), 0.04316493209409391), ((0, 3), 0.02437329621795089), ((0, 4), 0.03415714638246207), ((0, 5), 0.007133488057056619), ((0, 6), 0.019162468447168377), ((0, 7), 0.0797067840787825), ((0, 8), 0.031457121330428564), ((0, 9), 0.045623307833974135), ((0, 10), 0.02147379296416174), ((0, 11), 0.03644718158569349), ((0, 12), 0.06450593403406302), ((0, 13), 0.005598262400959731), ((0, 14), 0.0289555047757694), ((0, 15), 0.07050958926506035), ((0, 16), 0.02752328179667658), ((0, 17), 0.06974589752032252), ((0, 18), 0.07901765192101934), ((0, 19), 0.0636633375507631), ((0, 20), 0.03289432894894885), ((0, 21), 0.034456460457243474), ((0, 22), 0.08295971828416247), ((0, 23), 0.06396460994377069), ((0, 24), 0.08381654078878188), ((0, 25), 0.013123405732564701), ((0, 26), 0.01460692516401438), ((0, 27), 0.06039711901613763), ((0, 28), 0.006641752060098471), ((0, 29), 0.03757581516

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [46]:
import numpy as np


ROWS_A = 2
COLS_A = 3
COLS_B = 4 * 10

small_matrix = np.random.rand(ROWS_A, COLS_A)
large_matrix = np.random.rand(COLS_A, COLS_B)


reference_solution = np.matmul(small_matrix, large_matrix)

def flatten(nested_iterable):

    for iterable in nested_iterable:
        for element in iterable:
            yield element

def group_by_key(iterable):

    grouped = {}
    for (key, value) in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

def group_by_key_distributed(map_partitions, partitioner):

    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (key, value) in map_partition:
            partition_id = partitioner(key)
            partitions[partition_id][key] = partitions[partition_id].get(key, []) + [value]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in enumerate(partitions)]

def partitioner(obj):

    return hash(obj) % reducers

def MapReduceDistributed(input_format, map_func, reduce_func, partitioner=partitioner, combiner=None):

    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: map_func(*k1v1), record_reader)), input_format())

    if combiner is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: combiner(*k2v2), group_by_key(map_partition))), map_partitions)

    reduce_partitions = group_by_key_distributed(map_partitions, partitioner)  # Shuffle

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: reduce_func(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(sum(len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions]))))
    return reduce_outputs

def as_matrix(reduce_output):

    reduce_output = list(reduce_output)
    num_rows = max(i for ((i, k), vw) in reduce_output) + 1
    num_cols = max(k for ((i, k), vw) in reduce_output) + 1
    result_matrix = np.empty(shape=(num_rows, num_cols))
    
    for ((i, k), vw) in reduce_output:
        result_matrix[i, k] = vw
    return result_matrix

def input_format():

    first_matrix_data = []
    for i in range(small_matrix.shape[0]):
        for j in range(small_matrix.shape[1]):
            first_matrix_data.append(((0, i, j), small_matrix[i, j]))

    global num_maps
    split_size = int(np.ceil(len(first_matrix_data) / num_maps))

    for i in range(0, len(first_matrix_data), split_size):
        yield first_matrix_data[i:i + split_size]

    second_matrix_data = []
    for j in range(large_matrix.shape[0]):
        for k in range(large_matrix.shape[1]):
            second_matrix_data.append(((1, j, k), large_matrix[j, k]))

    split_size = int(np.ceil(len(second_matrix_data) / num_maps))

    for i in range(0, len(second_matrix_data), split_size):
        yield second_matrix_data[i:i + split_size]

def map_join(key1, value1):

    matrix_id, i, j = key1
    value = value1

    if matrix_id == 0:
        yield (j, (matrix_id, i, value))
    else:
        yield (i, (matrix_id, j, value))

def reduce_join(key, values):

    from_first_matrix = [v for v in values if v[0] == 0]
    from_second_matrix = [v for v in values if v[0] == 1]

    for f in from_first_matrix:
        for s in from_second_matrix:
            yield ((f[1], s[1]), f[2] * s[2])

def get_joined():

    for j in joined:
        print("Join Part:", j)
        yield j[1]

def map_multiply(key1, value1):

    yield (key1, value1)

def reduce_multiply(key, values):

    yield (key, sum(values))


num_maps = 3
reducers = 2


partitioned_output = MapReduceDistributed(input_format, map_join, reduce_join)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(get_joined, map_multiply, reduce_multiply)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)


solution_values = []
for p in pre_result:
    solution_values.extend(p[1])

print(solution_values)


is_close = np.allclose(reference_solution, as_matrix(solution_values))
print("Are the results close? ", is_close)


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.5561379275762566), ((0, 1), 0.9050374868937633), ((0, 2), 0.6719642684398045), ((0, 3), 0.8031503706107552), ((0, 4), 0.6307411118613933), ((0, 5), 0.7787424962058122), ((0, 6), 0.2989597324641228), ((0, 7), 0.8336061824928019), ((0, 8), 0.8479479540392121), ((0, 9), 0.029018698455134828), ((0, 10), 0.6533675311338043), ((0, 11), 0.47821014425993574), ((0, 12), 0.32484451685365134), ((0, 13), 0.7908334208373918), ((0, 14), 0.4534870410038086), ((0, 15), 0.08903611535336224), ((0, 16), 0.388567951850718), ((0, 17), 0.5408466659144899), ((0, 18), 0.7136350003880562), ((0, 19), 0.7303988350056633), ((0, 20), 0.6244750561081965), ((0, 21), 0.07960772755411526), ((0, 22), 0.07520998786697089), ((0, 23), 0.40240666007359255), ((0, 24), 0.14780704997587749), ((0, 25), 0.6324184290681142), ((0, 26), 0.39596781804804454), ((0, 27), 0.12143627398298609), ((0, 28), 0.20797557646917614), ((0, 29), 0.42756346113807275), ((0, 30), 0.0516